![Banner](03_jupyter/banner.png)

## Vorbereitung: Einrichtung der Umgebung und Verbindung zur Oracle-Datenbank

Zu Beginn wird die Arbeitsumgebung eingerichtet und eine Verbindung zu einer Oracle-Datenbank hergestellt. Dies umfasst die Installation der notwendigen Python-Pakete und die Initialisierung des Oracle-Clients.

#### Installierte Pakete:
- `oracledb`: Wird benötigt, um eine Verbindung zu Oracle-Datenbanken herzustellen und SQL-Abfragen auszuführen.
- `ipython-sql`: Ermöglicht die Verwendung von SQL in Jupyter-Notebooks.
- `pandas`: Dient der Datenverarbeitung und -analyse. Hier wird es verwendet, um die Ergebnisse von SQL-Abfragen in DataFrames zu laden und zu verwalten.

#### Code-Details:
- Der Oracle-Client wird mit `oracledb.init_oracle_client` initialisiert, wobei das Verzeichnis für die Oracle-Bibliotheken angegeben wird.
- Die Umgebungsvariablen `HOST_NAME` und `PDB_NAME` werden genutzt, um den Connection String (`dsn`) für die Oracle-Datenbank zu erstellen. Diese enthält den Hostnamen und den Pluggable Database Name (PDB), die benötigt werden, um die Datenbank anzusprechen.
- Es wird eine Verbindung zur Oracle-Datenbank mit dem Benutzer `vector` und dem Passwort `vector` aufgebaut. Die Verbindungszeichenfolge kombiniert Host und PDB, die in den Umgebungsvariablen hinterlegt sind.


In [1]:
!pip install oracledb 
!pip install ipython-sql
!pip install pandas

## Datenbank Verbindung erzeugen
In diesem Beispiel wird eine Autonomous Database benutzt. Die verwendeten Packages DBMS_CLOUD, DBMS_CLOUD_AI und so weiter sind mittlerweile auch auf Oracle23ai und Oracle 19c nachinstallierbar.
https://oracle-base.com/articles/21c/dbms_cloud-installation


In [26]:
import oracledb
import pandas as pd
import os
import warnings
import time

warnings.filterwarnings('ignore')
pd.set_option('expand_frame_repr', False)
pd.options.display.max_colwidth = 800

d = '/home/jovyan/.jupyter/instantclient_23_5'
oracledb.init_oracle_client(lib_dir=d)
cs = '(description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1521)(host=adb.eu-frankfurt-1.oraclecloud.com))(connect_data=(service_name=xxxyyyyzzzzz_myatp23ai_low.adb.oraclecloud.com))(security=(ssl_server_dn_match=yes)))'

print(cs)
# should be something like 'db23ai.subbb3fff175.quickcluster.oraclevcn.com/michael.subbb3fff175.quickcluster.oraclevcn.com'

(description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1521)(host=adb.eu-frankfurt-1.oraclecloud.com))(connect_data=(service_name=gfde677d3a923a9_atp23ai_low.adb.oraclecloud.com))(security=(ssl_server_dn_match=yes)))


In [27]:
connection = oracledb.connect(user='ADMIN', password='Welcome1234#', dsn=cs)
print(connection)

<oracledb.Connection to admin@(description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1521)(host=adb.eu-frankfurt-1.oraclecloud.com))(connect_data=(service_name=xxxyyyzzz_myatp23ai_low.adb.oraclecloud.com))(security=(ssl_server_dn_match=yes)))>


## Vorab: Benutzer den Netzwerk Zugang zur KI erlauben
Sollte als admin-Benutzer erfolgen, daher nur zu Dokumentationszwecken

## Credentials fuer den KI Zugang hinterlegen

In [29]:
sql = """
BEGIN
   DBMS_CLOUD.CREATE_CREDENTIAL(
        credential_name => 'OLLAMA_CRED',
        username    => 'OLLAMA',
        password    => 'OLLAMA'
   );
END;
"""



tic = time.perf_counter()
with connection.cursor() as cursor:     
    cursor.execute(sql)
    if cursor.warning :
        print(cursor.warning)
    else :
        print("SQL execution successful")
toc = time.perf_counter()
print(f"Loading model took {toc - tic:0.4f} seconds")


SQL execution successful
Loading model took 0.2706 seconds


## Erstellen des Profils fuer den KI Zugang



In [60]:
sql= """
BEGIN
   DBMS_CLOUD_AI.CREATE_PROFILE(
        profile_name =>'OLLAMA',
        attributes   =>'{
          "credential_name": "ollama_cred",
          "comments":              "true", 
          "constraints":           "true", 
          "annotations":           "true", 
          "object_list": [{"owner": "SH", "name": "customers"},                
                        {"owner": "SH", "name": "countries"},                
                        {"owner": "SH", "name": "supplementary_demographics"},
                        {"owner": "SH", "name": "profits"},                  
                        {"owner": "SH", "name": "promotions"},   
                        {"owner": "SH", "name": "sales"},   
                        {"owner": "SH", "name": "products"}],
           "model" : "codegemma:7b",
           "provider_endpoint": "var.130-61-137-234.nip.io",
           "conversation" : "true"
        }');
END;
"""


tic = time.perf_counter()
with connection.cursor() as cursor:     
    cursor.execute(sql)
    if cursor.warning :
        print(cursor.warning)
    else :
        print("SQL execution successful")
toc = time.perf_counter()
print(f"Loading model took {toc - tic:0.4f} seconds")



SQL execution successful
Loading model took 0.6123 seconds


## Profil aktivieren


In [61]:
sql= """
BEGIN
   DBMS_CLOUD_AI.SET_PROFILE('OLLAMA');
END;
"""

tic = time.perf_counter()
with connection.cursor() as cursor:     
    cursor.execute(sql)
    if cursor.warning :
        print(cursor.warning)
    else :
        print("SQL execution successful")
toc = time.perf_counter()
print(f"Loading model took {toc - tic:0.4f} seconds")

SQL execution successful
Loading model took 0.0395 seconds


## SQL Generierung testen
Mittels neuer "select ai" syntax

In [62]:
sql= """
select ai how many customers exist;
"""

df = pd.read_sql(sql=sql, con=connection)
display(df)

,TOTAL_CUSTOMERS
0,55500


## SQL Generierung testen
Mittels DBMS_CLOUD_AI.GENERATE

In [63]:
sql="""
SELECT DBMS_CLOUD_AI.GENERATE(prompt       => 'how many customers',
                              profile_name => 'OLLAMA',
                              action       => 'runsql') result
FROM dual
"""

df = pd.read_sql(sql=sql, con=connection)
display(df)

,RESULT
0,"[\n {\n ""TOTAL_CUSTOMERS"" : 55500\n }\n]"


In [40]:
sql="""
SELECT DBMS_CLOUD_AI.GENERATE(prompt       => 'how many customers',
                              profile_name => 'OLLAMA',
                              action       => 'showsql') statement
FROM dual
"""

df = pd.read_sql(sql=sql, con=connection)
display(df)

,STATEMENT
0,SELECT COUNT(*) AS total_customers\nFROM SH.CUSTOMERS


In [42]:
sql="""
SELECT DBMS_CLOUD_AI.GENERATE(prompt       => 'how many customers',
                              profile_name => 'OLLAMA',
                              action       => 'explainsql') explain
FROM dual
"""

df = pd.read_sql(sql=sql, con=connection)
display(df)

,EXPLAIN
0,```sql\nSELECT COUNT(*) AS total_customers\nFROM SH.CUSTOMERS;\n```


In [45]:
sql="""
SELECT DBMS_CLOUD_AI.GENERATE(prompt       => 'how many customers',
                              profile_name => 'OLLAMA',
                              action       => 'narrate') descriptive
FROM dual
"""

df = pd.read_sql(sql=sql, con=connection)
display(df)

,DESCRIPTIVE
0,"The result shows that there are a total of 55,500 customers in the database."


In [47]:
sql="""
SELECT DBMS_CLOUD_AI.GENERATE(prompt       => 'what is oracle autonomous database',
                              profile_name => 'OLLAMA',
                              action       => 'chat') chat
FROM dual
"""

df = pd.read_sql(sql=sql, con=connection)
display(df)

,CHAT
0,"**Oracle Autonomous Database**\n\nOracle Autonomous Database is a cloud-based, fully managed database service that eliminates the need for manual database administration. It is a self-service, pay-as-you-go database that provides a secure, scalable, and high-performance environment for applications."


In [66]:
sql="""
SELECT DBMS_CLOUD_AI.GENERATE(prompt       => 'which 10 customers have spent the most money , grouped by regions in the countries table',
                              profile_name => 'OLLAMA',
                              action       => 'showsql')
FROM dual
"""

df = pd.read_sql(sql=sql, con=connection)
display(df)

,"DBMS_CLOUD_AI.GENERATE(PROMPT=>'WHICH10CUSTOMERSHAVESPENTTHEMOSTMONEY,GROUPEDBYREGIONSINTHECOUNTRIESTABLE',PROFILE_NAME=>'OLLAMA',ACTION=>'SHOWSQL')"
0,"Sorry, unfortunately a valid SELECT statement could not be generated for your natural language prompt. Here is some more information to help you further: \n\n```sql\nSELECT C.CUST_COUNTRY, C.CUST_LAST_NAME || ', ' || C.CUST_FIRST_NAME AS CUSTOMER_NAME,\n SUM(S.AMOUNT_SOLD) AS TOTAL_SPENT\nFROM SH.CUSTOMERS C\nJOIN SH.SALES S ON C.CUST_ID = S.CUST_ID\nJOIN SH.COUNTRIES CO ON C.COUNTRY_ID = CO.COUNTRY_ID\nGROUP BY C.CUST_COUNTRY, C.CUST_LAST_NAME, C.CUST_FIRST_NAME\nORDER BY TOTAL_SPENT DESC\nFETCH FIRST 10 ROWS ONLY\n\nException encountered: ORA-00904: ""C"".""CUST_COUNTRY"": invalid identifier"
